
<div align="center">
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/f/fb/Escudo-UdeA.svg/970px-Escudo-UdeA.svg.png" width="300" height="300">
</div>

# **MODELO DE PREDICCIÓN DE TIEMPO DE ESPERA DE LOS ASEGURADOS EN INCIDENTES DE TRÁNSITO**

# **ESTUDIANTES**

# **Diego Fernando Londoño Londoño**

# **Yenny Patricia Vergara Monsalve**

# **1. DESCRIPCIÓN DEL PROYECTO**


El presente proyecto tiene como objetivo desarrollar un modelo predictivo que estime el tiempo de espera de un asegurado tras reportar un accidente de tránsito, con el fin de optimizar los procesos de atención y mejorar la experiencia del cliente en una compañía aseguradora.

La iniciativa surge de la necesidad de contar con herramientas analíticas que permitan anticipar demoras y gestionar de forma eficiente los recursos disponibles en campo, como gestores de siniestros y unidades móviles.

Para ello, se utilizó un conjunto de datos históricos de siniestros, registrados por la aseguradora a través de sus sistemas de gestión durante un período determinado. Estos datos incluyen variables relacionadas con la ubicación, hora del incidente, tipo de accidente, tráfico, disponibilidad de personal y otros factores relevantes como el clima y la distancia.

Se implementó un proceso iterativo que incluyó limpieza, análisis exploratorio, selección de variables y entrenamiento de distintos modelos de machine learning como regresión lineal, árboles de decisión y random forest, evaluando su desempeño mediante métricas como el MAE y el RMSE.

Entre los principales obstáculos se encontraron problemas de calidad de datos, presencia de valores atípicos y desequilibrios en la distribución de los tiempos de atención.

Los resultados obtenidos muestran que el modelo tiene un desempeño aceptable para predecir tiempos de espera con un margen de error razonable, y representa una herramienta valiosa para la toma de decisiones operativas en tiempo real.


# **2. OBJETIVOS**


**Objetivo del Proyecto**

Predecir el tiempo de espera en la atención de un asegurado durante una incidencia de tránsito, desarrollando un modelo de Machine Learning con el fin de brindar soporte al proceso del gestor de seguros.

**Objetivos Específicos**

•	Realizar un análisis exploratorio y limpieza de datos provenientes de los datos históricos de siniestros para identificar patrones, inconsistencias y variables relevantes.

•	Construir variables derivadas (ingeniería de características) que integren factores espaciales, temporales y contextuales como clima, tráfico, distancia, entre otros.

•	Entrenar y comparar modelos de aprendizaje automático para estimar el tiempo de espera, evaluando su desempeño con métricas como MAE, RMSE y R2

•	Seleccionar el modelo con mejor desempeño y validar su capacidad predictiva para su futura integración en los sistemas de gestión de siniestros de la aseguradora.

•	Emitir el conjunto de recomendaciones correspondientes que permitan mejorar el rendimiento del modelo seleccionado en etapas posteriores de la investigación



# **3. IMPORTACIÓN DE LIBRERÍAS**

## **3.1. Librerias y configuraciones previas**

In [ ]:
# Gestion de librerias
# ==============================================================================
import numpy as np
import pandas as pd
import math
from importlib import reload

# Almacenar en caché los resultados de funciones en el disco
# ==============================================================================
import joblib

# Preprocesado y modelado
# ==============================================================================

#creación de modelos
from sklearn.ensemble import GradientBoostingRegressor

#Escalar Variables
from sklearn.preprocessing import MinMaxScaler

#evaluacion de variables
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterGrid
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_absolute_error

# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')
import multiprocessing

# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
from rich import print

# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')

#Instalar libereria
!pip install category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.9/85.9 kB 2.4 MB/s eta 0:00:00


# **4. CARGA DE DATOS**

## **4.1. Carga del dataset**

In [ ]:
# Cargar dataset desde un archivo de Excel
# ==============================================================================
import pandas as pd

archivo = "https://github.com/ferdilo04/Monografia_EACD/raw/refs/heads/main/DataSet/DataSet_aseguradora.xlsx"
df = pd.read_excel(archivo)

# Ver estructura básica
print("Dimensiones del dataset:", df.shape)
print("\nPrimeras filas:")
display(df.head(3))

# Configuración estética
sns.set(style="whitegrid")
plt.figure(figsize=(16, 10))

Dimensiones del dataset:
(14519, 29)

Primeras filas:

,idproceso,placa,fecha,fechallegada,annoregistro,mesregistro,diaregistro,horaregistro,diasemana,annoatencion,...,Instancia,clienteimportante,Acuerdo,UsuarioRegistra,LiberaVehiculo,ResultadoFallo,Aseguradora,Abogadounico,RandomAbogado,Decripcion
0,8812,USV110,2016-10-14 13:42:32.397,NaN,2016,10,14,13,Viernes,NaN,...,AUDIENCIA,NaN,NaN,Elkin Lezcano,NaN,NaN,Liberty Seguros SA,Andi Asistencia SA,Juan Felipe Hoyos Botero,Liberty <br>Audiencia <br>MIGUEL ALEJANDRO CRU...
1,10995,KAO616,2016-11-28 16:59:55.587,2016-11-28 17:00:22.177,2016,11,28,16,Lunes,2016.0,...,SIMPLE,NaN,NaN,JOHAN ALEXANDER MESA RIVERA,NaN,NaN,Liberty Seguros SA,Luis Fernando Saldarriaga Rodriguez,Elkin Lezcano,LIBERTY<br>PRELIMINAR LESIONES<br>JENY QUINTER...
2,10998,HNZ243,2016-11-29 10:47:56.770,2016-11-29 11:37:14.017,2016,11,29,10,Martes,2016.0,...,TELEFÓNICA,NaN,NaN,Elkin Lezcano,NaN,NaN,Liberty Seguros SA,Andi Asistencia SA,Luis Fernando Saldarriaga Rodriguez,Liberty. <br>Telefónica <br>BANCO DE OCCIDENTE...


<Figure size 1600x1000 with 0 Axes>

<Figure size 1600x1000 with 0 Axes>

## **4.2. Comprensión de los Datos**


Se realizó un análisis exploratorio sobre el conjunto de datos proporcionado por la empresa AsisNet, que contiene información detallada de accidentes de tránsito atendidos,los datos son proporcionados son partir del **mes de abril del 2016 al 11 de mayo del 2025**.

El DataSet tiene el nombre **DataSet_PREDITIMESACC.xlsx** y se encuentra en formato de excel con extensión .xlsx , con un tamaño de **3.330 Kb** el cual contiene **14519** registros y **29** variables , la información del dataset fue obtenida mediante una consulta Transat SQL en un servidor de base de datos SQL Server, que contiene información de las incidencias de tránsito y de las gestiones de atención realizadas por los agentes de las aseguradoras suscritas a la empresa AsisNet .


**A continuación una descripción de las variables:**

****

| Columna           | Descripción                                                                 | Ejemplo                     |
|-------------------|------------------------------------------------------------------------------|-----------------------------|
| idproceso         | Código para identificar el caso                                              | 10                          |
| placa             | Placa del vehículo                                                           | BQX320                      |
| fecha             | Fecha en que se reporta el accidente                                         | 2025-05-02 10:40:25         |
| fechallegada      | Fecha en que se llega al sitio donde ocurrió el accidente                   | 2025-05-02 10:50:25         |
| annoregistro      | Año en que se reporta el accidente                                           | 2025                        |
| mesregistro       | Mes en que se reporta el accidente                                           | 5                           |
| diaregistro       | Día en que se reporta el accidente                                           | 2                           |
| horaregistro      | Hora en que se reporta el accidente                                          | 10                          |
| diasemana         | Día de la semana en que se reporta el accidente                              | Lunes                       |
| annoatencion      | Año en que se llega al sitio donde ocurrió el accidente                     | 2025                        |
| mesatencion       | Mes en que se llega al sitio donde ocurrió el accidente                     | 5                           |
| diaatencion       | Día en que se llega al sitio donde ocurrió el accidente                     | 2                           |
| Horaantencion     | Hora en que se llega al sitio donde ocurrió el accidente                    | 10                          |
| TiempoAtencion    | Diferencia en minutos entre el reporte y la llegada                         |                             |
| HoraPicoTarde     | ¿Ocurrió en hora pico de la tarde (5 a 7 pm)? (Sí/No)                       | Si                          |
| HoraPicoManana    | ¿Ocurrió en hora pico de la mañana (6:30 a 8:30 am)? (Sí/No)                | No                          |
| InicioNoche       | ¿Ocurrió entre las 7 pm y 11:59 pm? (Sí/No)                                 | No                          |
| Amanecer          | ¿Ocurrió entre las 12 am y 5:59 am? (Sí/No)                                 | Si                          |
| Municipio         | Municipio donde ocurrió el accidente                                         | No                          |
| Intsancia         | Tipo de accidente reportado por la aseguradora                              | Preliminar con lesiones     |
| clienteimportante | Si el asegurado es cliente importante                                       | Banco de accidente          |
| Acuerdo           | Al acuerdo que se llega entre las partes implicadas                         | Tránsito                    |
| UsuarioRegistra   | Usuario que registra el caso en el sistema                                   | Carolina Garcia Valencia    |
| LiberaVehiculo    | Si el vehículo fue retenido y luego liberado                                | Liberado                    |
| ResultadoFallo    | Si el fallo en audiencias es a favor o en contra                            | A favor                     |
| Aseguradora       | Aseguradora que reporta el accidente                                        | Liberty Seguros             |
| Abogadounico      | Abogado disponible para atender el caso                                     | Carolina Garcia Valencia    |
| RandomAbogado     | Abogado que finalmente atiende el caso                                      | Carolina Garcia Valencia    |
| Descripción     | Detalle de caso                                      | SIMPLE <br>PREVISORA LIVIANOS <br>MEDELLIN <br>7516811<br>JUAN CARLOS PEREZ <br>3004931460<br>MTT750<br>CARRERA 46 CON CALLE 78 24 <br>CAMPO VALDEZ <br>ASIG 19:06<br><br>SERGIO BARRIENTOS <br>40 M <br>   |

## **4.3. Recolección de Datos Adicionales Clima - Coordenadas GPS - Distancia**

Identificar las fuentes de datos necesarias, para nuestro análisis requerimos de una aplicación para limpiar las dirección, tratamiento del Clima y API's para hallar distancias

###**4.3.1. Conexión con Api del clima**


La función resumen_clima se encarga de obtener un resumen binario de la precipitación (si llovió o no) para una ubicación geográfica específica en una fecha determinada, utilizando la librería Meteostat.

En esencia, la función determina si hubo precipitación (prcp > 0) o no (prcp = 0 o NA) en un día y lugar concreto, devolviendo "Si" o "No".

In [ ]:
#Instalar libreria para buscar el clima segun una fecha
!pip install meteostat

In [ ]:
from meteostat import Point, Daily
from datetime import datetime

def resumen_clima(municipio: str, lat: float, lon: float, fecha: str) -> str:
    """
    Devuelve un resumen del clima en el municipio y fecha indicada usando Meteostat.

    :param municipio: Nombre del municipio (solo para mostrar en el texto)
    :param lat: Latitud del municipio
    :param lon: Longitud del municipio
    :param fecha: Fecha en formato 'YYYY-MM-DD'
    :return: Texto con resumen del clima
    """
    # Crear ubicación
    location = Point(lat, lon)

    # Convertir fecha a datetime
    fecha_dt = datetime.strptime(fecha, "%Y-%m-%d")

    # Descargar datos diarios
    data = Daily(location, fecha_dt, fecha_dt)
    data = data.fetch()

    if data.empty:
        return f"NA"

    row = data.iloc[0]
    tavg = row["tavg"]
    tmin = row["tmin"]
    tmax = row["tmax"]
    prcp = row["prcp"]

    if pd.notna(prcp) and prcp > 0:
        resumen = f"Si"
    else:
        resumen = "No"

    return resumen

In [ ]:
#Llamar function para traer el estado del clima segun la fecha de la incidencia
def apply_resumen_clima(row):
    if pd.isna(row["annoatencion"]) or pd.isna(row["mesatencion"]) or pd.isna(row["diaatencion"]):
        return "NA"
    else:
        date_str = f"{int(row['annoatencion'])}-{int(row['mesatencion']):02d}-{int(row['diaatencion']):02d}"
        return resumen_clima("Medellín", 6.2518, -75.5636, date_str)

df["clima"] = df.apply(apply_resumen_clima, axis=1)

df.head(30)

,idproceso,placa,fecha,fechallegada,annoregistro,mesregistro,diaregistro,horaregistro,diasemana,annoatencion,...,UsuarioRegistra,LiberaVehiculo,ResultadoFallo,Aseguradora,Abogadounico,RandomAbogado,Decripcion,Lluvia,distancia,clima
0,1113,MOY664,2016-06-08 09:20:16.223,2016-06-08 09:20:16.223,2016,6,8,9,Miercoles,2016.0,...,Juan Felipe Hoyos Botero,NaN,NaN,Liberty Seguros SA,Juan Felipe Hoyos Botero,ELSA ROSA VEGA LOPEZ,Liberty <br>PRELIMINAR <br>LESIONES <br>ASIG 0...,No,8482.615228,No
1,1114,TSZ179,2016-06-08 11:56:02.857,2016-06-08 11:56:02.857,2016,6,8,11,Miercoles,2016.0,...,Elkin Lezcano,NaN,NaN,Asegurado,Juan Felipe Hoyos Botero,Juan Felipe Hoyos Botero,Liberty <br>Lesiones <br>Andres GARCES <br>313...,No,8486.581225,No
2,1115,STW598,2016-06-08 14:03:49.283,2016-06-08 14:03:49.283,2016,6,8,14,Miercoles,2016.0,...,Elkin Lezcano,NaN,NaN,Asegurado,Juan Felipe Hoyos Botero,ELSA ROSA VEGA LOPEZ,Liberty <br>SIMPLE <br>Jorge VELEZ <br>3192538...,No,8482.832639,No
3,1116,TSE057,2016-06-08 18:29:41.737,2016-06-08 18:29:41.737,2016,6,8,18,Miercoles,2016.0,...,Elkin Lezcano,NaN,NaN,Asegurado,Juan Felipe Hoyos Botero,Carolina Garcia Valencia,ATENDIO JHON JAIRO TABARES <br><br>Liberty <br...,No,8639.194598,No
4,1117,USX686,2016-06-08 19:46:18.653,2016-06-08 19:46:18.653,2016,6,8,19,Miercoles,2016.0,...,Elkin Lezcano,NaN,NaN,Asegurado,Juan Felipe Hoyos Botero,SEBASTIAN LOPEZ GOMEZ,Liberty <br>SIMPLE <br>Calle 33 x av regional ...,No,8482.415758,No
5,1119,FGK906,2016-06-09 09:06:27.883,2016-06-09 09:06:27.883,2016,6,9,9,Jueves,2016.0,...,Juan Felipe Hoyos Botero,NaN,NaN,Asegurado,Juan Felipe Hoyos Botero,SEBASTIAN LOPEZ GOMEZ,FGK906<br>Fabio ZULUAGA <br>3104361803<br>Envi...,No,8488.610627,No
6,1120,TAM116,2016-06-09 15:48:31.240,2016-06-09 15:48:31.240,2016,6,9,15,Jueves,2016.0,...,Juan Felipe Hoyos Botero,NaN,NaN,Asegurado,Juan Felipe Hoyos Botero,Juan Felipe Hoyos Botero,Liber Marín<br>3124053102<br>Cra 50 a calle 82...,No,8843.658737,No
7,1121,IEX392,2016-06-09 17:37:54.000,2016-06-09 17:37:54.000,2016,6,9,17,Jueves,2016.0,...,Elkin Lezcano,NaN,NaN,Asegurado,Juan Felipe Hoyos Botero,ELSA ROSA VEGA LOPEZ,Liberty <br>SIMPLE <br>Iex392 <br>3103888543<b...,No,8486.238007,No
8,1122,INN976,2016-06-09 18:06:08.560,2016-06-09 18:06:08.560,2016,6,9,18,Jueves,2016.0,...,Elkin Lezcano,NaN,NaN,Asegurado,Juan Felipe Hoyos Botero,JOHAN ALEXANDER MESA RIVERA,Liberty <br>SIMPLE <br>Banco de occidente <br>...,No,8486.803291,No
9,1126,BLX520,2016-06-10 14:54:37.880,2016-06-10 14:54:37.880,2016,6,10,14,Viernes,2016.0,...,Juan Felipe Hoyos Botero,NaN,NaN,Liberty Seguros SA,Luis Fernando Saldarriaga Rodriguez,John Jairo Tabares Chavarriaga,Liberty <br>PRELIMINAR <br>LESIONES <br>BLX520...,No,8489.210638,No


In [ ]:
# Guardar datase con la informacion del clima
df.to_excel("DataSet_PREDITIMESACC_V1.xlsx", index=False)

###**4.3.2. Script para limpiar direcciones**

El script nos permite sustituir texto con expresiones regulares, modificar valores, dejando el campo descripción limpio con direcciones que se pueden geolocalizar

In [ ]:
import pandas as pd
import re

# Subir tu archivo Excel desde tu PC
from google.colab import files
#uploaded = files.upload()

# Reemplaza el nombre del archivo si es distinto
ruta = "https://github.com/ferdilo04/Monografia_EACD/raw/refs/heads/main/DataSet/DataSet_PREDITIMESACC_V1.xlsx"
df = pd.read_excel(ruta)

# Leer la hoja "Hoja"
#df = pd.read_excel(ruta, sheet_name="Hoja")

# --- CONFIGURACIÓN ---
BARRIOS_CIUDADES = [
    "MEDELLÍN", "MEDELLIN", "BELÉN", "BELEN", "POBLADO", "ENVIGADO",
    "ITAGÜÍ", "ITAGUI", "SABANETA", "COPACABANA", "LA ESTRELLA",
    "SAN DIEGO", "BELLO", "CALDAS", "BARRIO", "CASTILLA", "MANRIQUE",
    "ROBLEDO", "ARANJUEZ"
]

nombre_persona_regex = re.compile(r"^[A-ZÁÉÍÓÚÑ][a-záéíóúñ]+(?:\s+[A-ZÁÉÍÓÚÑ][a-záéíóúñ]+)+$")
patron_prefijo_dir = re.compile(r"^(?:K|KR|CRA|CR|CL|CALLE|CARRERA|AV|AUTOPISTA|DIAG|TRANSV|TRV)\b", re.IGNORECASE)
patron_direccion = re.compile(
    r"("
    r"(?:(?:K|KR|CRA|CR|CL|CALLE|CARRERA|AV|AUTOPISTA|DIAG|TRANSV|TRV)\s*[\dA-Z\-\.\#]+(?:\s*(?:CON|No\.|NO|#|\-|\s)\s*[\dA-Z\s\-]*){0,3})"
    r"|(?:\b\d{1,3}\s+[A-Z]?\s*\d{1,3}\s+[A-Z]?\s*\d{0,3}\b)"
    r"|(?:\b(?:CERCA|FRENTE|SECTOR|ALTURA|ENTRADA|SALIDA|SENTIDO|ESQUINA|ANTES\s*DE|DESPU[EÉ]S\s*DE|HACIA|COSTADO|AL\s+FRENTE|BOMBA|CIRCULAR|AL\s+PIE|DEBAJO\s+DE|ENTRE\s+LA|CORREGIMIENTO|ESTACION|PARQUE|GLORIETA|TUNEL|ROMPOINT|CENTRO\s+COMERCIAL|Soterrado)\b[\w\s\-\#\,\.]{3,100})"
    r")", re.IGNORECASE
)

def extraer_ubicacion_avanzada(texto):
    if pd.isna(texto):
        return None
    s = str(texto).replace("<br/>", "\n").replace("<br>", "\n").replace("<BR>", "\n")
    lineas = [l.strip() for l in s.split("\n") if l.strip()]

    ciudad = None
    for ln in lineas:
        for barrio in BARRIOS_CIUDADES:
            if re.search(r"\b" + re.escape(barrio) + r"\b", ln, re.IGNORECASE):
                ciudad = barrio.upper()
                break
        if ciudad: break

    candidatos = []
    for ln in lineas:
        if patron_prefijo_dir.search(ln):
            candidatos.append(("prefijo", ln))
    if not candidatos:
        for ln in lineas:
            if patron_direccion.search(ln):
                candidatos.append(("pattern", ln))
    if not candidatos:
        for ln in lineas:
            if re.search(r"\b\d{1,3}\s+[A-Z]?\s*\d{1,3}\b", ln):
                if len(re.sub(r"\D", "", ln)) < 7:
                    candidatos.append(("numeric_seq", ln))

    candidatos_filtrados = [(t, l) for t, l in candidatos if not nombre_persona_regex.match(l)]
    if candidatos_filtrados:
        prioridad = {"prefijo": 0, "pattern": 1, "numeric_seq": 2}
        candidatos_filtrados.sort(key=lambda x: (prioridad.get(x[0], 99), -len(x[1])))
        ubic = candidatos_filtrados[0][1]
    else:
        ubic = None

    if ciudad and ubic:
        return f"{ciudad} - {ubic}"
    elif ubic:
        return ubic
    elif ciudad:
        return ciudad
    else:
        return None

# Crear la nueva columna
df["UbicacionExtraida"] = df["Decripcion"].apply(extraer_ubicacion_avanzada)

# Guardar el nuevo archivo
nombre_salida = "DataSet_PREDITIMESACC_V2.xlsx"
df.to_excel(nombre_salida, index=False)

# Descargar el resultado
files.download(nombre_salida)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

###**4.3.3. Script para georeferenciar direcciones**

El script toma los valores de la descirpción del dataset, toma la dirección limpia del campo descripción, geolocaliza esa dirección para obtener sus coordenadas GPS y, de forma separada, filtra y exporta los datos correspondientes al municipio de "Medellín".

In [ ]:
from geopy.geocoders import Photon
import time
import os

def geocodificar_con_pausas(df, archivo_salida="resultado_coordenadas.xlsx"):
    geolocator = Photon(user_agent="mi_app", timeout=10)

    # Configuración de pausas
    PAUSA_CORTA = 1  # segundos
    PAUSA_LARGA_CADA = 50
    PAUSA_LARGA = 10

    #latitudes = []
    #longitudes = []
    latitudes = [None] * len(df)
    longitudes = [None] * len(df)

    print(f"Procesando {len(df)} direcciones...")

    for i, row in df.iterrows():
        direccion = f"{row['Direccion']}, {row['Municipio']}, Colombia"

        try:
            loc = geolocator.geocode(direccion)
            if loc:
                #latitudes.append(loc.latitude)
                #longitudes.append(loc.longitude)
                latitudes[i] = loc.latitude
                longitudes[i] = loc.longitude
                print(f"{i+1:03d}/✓: {direccion[:50]}...Latitud: {latitudes[i]}, Lonigtud: {longitudes[i]}...")
            else:
                #latitudes.append(None)
                #longitudes.append(None)
                latitudes[i] = None
                longitudes[i] = None
                print(f"{i+1:03d}/✗: {direccion[:50]}...")

        except Exception as e:
            #latitudes.append(None)
            #longitudes.append(None)
            latitudes[i] = None
            longitudes[i] = None
            print(f"{i+1:03d}/E: {direccion[:50]}... → {e}")

        # Pausas inteligentes
        if (i + 1) % PAUSA_LARGA_CADA == 0 and (i + 1) < len(df):
            print(f"  Pausa de {PAUSA_LARGA}s...")

            df_temp = df.copy()
            df_temp["latitud"] = latitudes
            df_temp["longitud"] = longitudes

            # Guardar progreso
            df_temp.to_excel(archivo_salida, index=False)
            progreso = ((i + 1) / len(df)) * 100
            print(f" GUARDADO AUTOMÁTICO: {i + 1}/{len(df)} filas ({progreso:.1f}%)")

            time.sleep(PAUSA_LARGA)
        else:
            time.sleep(PAUSA_CORTA)

    # Añadir resultados
    df_resultado = df.copy()
    df_resultado["latitud"] = latitudes
    df_resultado["longitud"] = longitudes

    # Guardar
    df_resultado.to_excel(archivo_salida, index=False)
    print(f" Guardado en: {archivo_salida}")

    return df_resultado

# Uso de la función
ruta = "/content/DataSet_PREDITIMESACC_V2.xlsx"
df = pd.read_excel(ruta)

archivo_salida = "DataSet_PREDITIMESACC_V3.xlsx"
resultado = geocodificar_con_pausas(df, archivo_salida)

# Estadísticas finales
exitosos = resultado["latitud"].notna().sum()
print(f"\ PROCESO COMPLETADO")
print(f" Guardado en: {archivo_salida}")
print(f" Total: {len(resultado)}")
print(f" Exitosos: {exitosos} ({exitosos/len(resultado)*100:.1f}%)")
print(f" Fallidos: {len(resultado) - exitosos}")

###**4.3.4. Script para calcular distancias**

El script nos permite calcular las distancias entre dos puntos(una coordenada de origen(latitud longitud) y una coordenada destino(latitud longitud)) utilizando la function vectorizada **vectorizada Haversine**


In [ ]:
# Configura la ruta y nombres de columnas ---
ruta_entrada = "/content/DataSet_PREDITIMESACC_V3.xlsx"
ruta_salida  = "DataSet_PREDITIMESACC_V4.xlsx"

lat_or_col = "latitud_o"    # columna latitud origen
lon_or_col = "longitud_o"    # columna longitud origen
lat_dest_col = "latitud_d" # columna latitud destino
lon_dest_col = "longitud_d" # columna longitud destino

# Cargar datos
df = pd.read_excel(ruta_entrada, sheet_name="Datos")

# Validar que existan las columnas
for col in [lat_or_col, lon_or_col, lat_dest_col, lon_dest_col]:
    if col not in df.columns:
        raise ValueError(f"No se encontró la columna '{col}' en el archivo. Columnas disponibles: {df.columns.tolist()}")

# Función vectorizada Haversine
def haversine_vectorized(lat1, lon1, lat2, lon2, radius=6371.0):
    """
    Devuelve distancia en kilómetros (por defecto).
    lat/lon en grados (arrays o Series).
    """
    # convertir grados a radianes
    lat1_rad = np.radians(lat1.astype(float))
    lon1_rad = np.radians(lon1.astype(float))
    lat2_rad = np.radians(lat2.astype(float))
    lon2_rad = np.radians(lon2.astype(float))

    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    a = np.sin(dlat / 2.0)**2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon / 2.0)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    d = radius * c
    return d

# Calcular distancias
# En kilómetros
df["dist_km"] = haversine_vectorized(df[lat_or_col], df[lon_or_col], df[lat_dest_col], df[lon_dest_col], radius=6371.0)

# En metros
df["dist_m"] = df["dist_km"] * 1000.0

# En millas (opcional)
df["dist_miles"] = df["dist_km"] * 0.621371


# Estadísticas
stats = {
    "count_valid": int(df["dist_km"].notna().sum()),
    "mean_km": float(df["dist_km"].mean()),
    "median_km": float(df["dist_km"].median()),
    "max_km": float(df["dist_km"].max()),
    "min_km": float(df["dist_km"].min())
}
print("Resumen de distancias (km):", stats)

# Guardar resultado de distancias en el dataset
df.to_excel(ruta_salida, index=False)
print(f"Archivo guardado: {ruta_salida}")
